In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
from datetime import datetime

# Plot
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Function for converting date in format dd/mm/yyy to yyyy-mm-dd datetime type
def convert_to_date(date_string):
    date_list = date_string.split("/")
    _day, _month, _year = date_list[0], date_list[1], date_list[2]
    date_str = _year+"-"+_month+"-"+_day
    return datetime.strptime(date_str, '%Y-%m-%d').date()

# Load weather data
data_weather = pd.read_csv('../raw_data/paris_weather.csv')
df_weather = data_weather

# Converting date in format dd/mm/yyy to yyyy-mm-dd datetime type
df_weather['date'] = df_weather['DATE'].apply(convert_to_date)
df_weather['date'] = pd.to_datetime(df_weather['date'])


# Creating day of week as a cyclical feature
# First, create the day of the week as a numerical feature
df_weather['day_of_week'] = pd.to_datetime(df_weather['date']).dt.weekday
# Since we have a 7 days week period (e.g., days in a week)
period = 7
# Convert 'day_of_week' to radians
df_weather['day_of_week_radians'] = 2 * np.pi * df_weather['day_of_week'] / period
# Create new features using sine and cosine
df_weather['day_of_week_sin'] = np.sin(df_weather['day_of_week_radians'])
df_weather['day_of_week_cos'] = np.cos(df_weather['day_of_week_radians'])
# Dropping ['month_radians']
df_weather.drop(columns=['day_of_week_radians', 'day_of_week', 'DATE'], inplace=True)


# Creates cyclical month feature according to the date
df_weather['month'] = df_weather.date.dt.month
# Assuming we have a 12 month period (e.g., month in a year)
period = 12
# Convert 'month' to radians
df_weather['month_radians'] = 2 * np.pi * df_weather['month'] / period
# Create new features using sine and cosine
df_weather['month_sin'] = np.sin(df_weather['month_radians'])
df_weather['month_cos'] = np.cos(df_weather['month_radians'])
# Dropping ['month_radians']
df_weather.drop(columns=['month_radians', 'month'], inplace=True)


# Generates periodical features for SUNRISE and SUNSET
# Creates cyclical time feature according to the SUNRISE time
df_weather['sunrise_time'] = pd.to_datetime(df_weather['SUNRISE']).dt.hour*60+pd.to_datetime(df_weather['SUNRISE']).dt.minute
# Assuming we have a 1440 minutes in period (e.g., minutes  in a day)
period = 1440
# Convert 'month' to radians
df_weather['sunrise_time_radians'] = 2 * np.pi * df_weather['sunrise_time'] / period
# Create new features using sine and cosine
df_weather['sunrise_time_sin'] = np.sin(df_weather['sunrise_time_radians'])
df_weather['sunrise_time_cos'] = np.cos(df_weather['sunrise_time_radians'])
# Dropping ['sunrise_time_radians']
df_weather.drop(columns=['sunrise_time_radians', 'sunrise_time', 'SUNRISE'], inplace=True)


# Creates cyclical time feature according to the SUNSET time
df_weather['sunset_time'] = pd.to_datetime(df_weather['SUNSET']).dt.hour*60+pd.to_datetime(df_weather['SUNSET']).dt.minute
# Assuming we have a 1440 minutes in period (e.g., minutes  in a day)
period = 1440
# Convert 'month' to radians
df_weather['sunset_time_radians'] = 2 * np.pi * df_weather['sunset_time'] / period
# Create new features using sine and cosine
df_weather['sunset_time_sin'] = np.sin(df_weather['sunset_time_radians'])
df_weather['sunset_time_cos'] = np.cos(df_weather['sunset_time_radians'])
# Dropping ['sunset_time_radians']
df_weather.drop(columns=['sunset_time_radians', 'sunset_time', 'SUNSET'], inplace=True)


# Setting new date column as index
df_weather.set_index(['date'], inplace=True)
df_weather.head()


# Creating the dictionary for the weather comments and converting OPINION to numbers
dict_meteo = {
'météo idéale': 4,
'météo favorable': 3,
'météo correcte': 2,
'météo défavorable': 1,
'météo très défavorable': 0
}
df_weather['opinion'] = df_weather['OPINION'].map(dict_meteo)
df_weather.drop(columns= ['OPINION'], inplace=True)
df_weather.head()


# Creates dictionary with Holidays
holidays = [
    '2021-1-1',
    '2021-4-5',
    '2021-5-1',
    '2021-5-8',
    '2021-5-13',
    '2021-5-24',
    '2021-7-14',
    '2021-8-15',
    '2021-11-1',
    '2021-11-11',
    '2021-12-25',
    '2022-1-1',
    '2022-4-18',
    '2022-5-1',
    '2022-5-8',
    '2022-5-26',
    '2022-6-6',
    '2022-7-14',
    '2022-8-15',
]
holidays = [pd.to_datetime(holiday)for holiday in holidays ]
# Add isHoliday column to the df_weather
df_weather['isHoliday'] = df_weather.index.map(lambda x: 1 if x in holidays else 0)
